# Sparkify Data Analysis with Cassandra

This project aims to create an ETL pipleline to process song play history files initially stored in CSV.  Part 1 of the project creates a large combined CSV file with columns to feed Apache Cassandra tables.  Part 2 creates tables to facilitate queries on the dataset and provides examples.

## Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the generated csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II. Apache Cassandra Data Model and Queries. 

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of the denormalized data in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
# Create an attribute dictionary for easier columnar reference to attributes
attr_dict = {
    'artist': 0,
    'first_name': 1,
    'gender': 2,
    'item_in_session': 3,
    'last_name': 4,
    'duration': 5,
    'level': 6,
    'location': 7,
    'session_id': 8,
    'song': 9,
    'user_id': 10
}

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [7]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify_music
WITH REPLICATION = 
{ 'class': 'SimpleStrategy', 'replication_factor': 1}
""")

#### Set Keyspace

In [8]:
session.set_keyspace('sparkify_music')

## Task #1
### Find the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Use session_id as parition key to spread sessions across nodes in cluster and
use item_in_session as clustering column to cluster the items in the session for faster reads

In [9]:
query = "CREATE TABLE IF NOT EXISTS song_by_item_in_session "
query = query + """
(session_id int, item_in_session int, artist text, song text, duration decimal,
PRIMARY KEY (session_id, item_in_session))
"""
session.execute(query)
                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_item_in_session (session_id, item_in_session, artist, song, duration)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[attr_dict['session_id']]),
                               int(line[attr_dict['item_in_session']]),
                               line[attr_dict['artist']],
                               line[attr_dict['song']],
                               float(line[attr_dict['duration']])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT artist, song, duration FROM song_by_item_in_session WHERE session_id = 338 AND item_in_session = 4"
rows = session.execute(query)

for row in rows:
    print(row.artist, row.song, row.duration)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Task #2
### Find the artists, songs (sorted by itemInSession) and user name (first and last name) for userid = 10, sessionid = 182

Use composite partition key of user_id and session_id to evenly spread users' sessions across nodes.  If only user_id were used a very active user
could cause a lot of writes to go to just one node.  Assuming sessions are of a fixed length, then using
the session_id should spread out the writes.
Use item_in_session to allow sorting of songs by item number in session.

    

In [12]:

query = "CREATE TABLE IF NOT EXISTS history_by_session "
query = query + """(
user_id int,
session_id int,
item_in_session int,
artist text,
song text,
first_name text,
last_name text,
PRIMARY KEY ((user_id, session_id), item_in_session)
)"""
session.execute(query)
                    
                    

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO history_by_session
        (user_id, session_id, item_in_session, artist, song, first_name, last_name)
        """
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[attr_dict['user_id']]),
                                int(line[attr_dict['session_id']]),
                                int(line[attr_dict['item_in_session']]),
                                line[attr_dict['artist']],
                                line[attr_dict['song']],
                                line[attr_dict['first_name']],
                                line[attr_dict['last_name']]))


In [14]:
query = """
SELECT artist, song, first_name, last_name FROM history_by_session
WHERE user_id = 10 AND session_id = 182 ORDER BY item_in_session
"""
rows = session.execute(query)

for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Task #3
### Find every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Use song as partition key with user_id clustering column to spread out songs uniformly across nodes, sort by user_id for fast retrieval, and provide a unique primary key for all song plays even if some users have the same full name.

In [15]:
query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + """(
song text,
user_id int,
first_name text,
last_name text,
PRIMARY KEY (song, user_id)
)"""
session.execute(query)
                    
                    

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[attr_dict['song']],
                                int(line[attr_dict['user_id']]),
                                line[attr_dict['first_name']],
                                line[attr_dict['last_name']]))



In [17]:
query = "SELECT first_name, last_name FROM users_by_song WHERE song = 'All Hands Against His Own'"
rows = session.execute(query)

for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
session.execute('DROP TABLE song_by_item_in_session')
session.execute('DROP TABLE history_by_session')
session.execute('DROP TABLE users_by_song')

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()